# Train Model

In [3]:
import pandas as pd
import numpy as np
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau

# ----------- GPU Configuration -----------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU(s) detected: {[gpu.name for gpu in gpus]}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Training on CPU.")

# ----------- 1. Load training and validation data -----------
train_df = pd.read_csv('./csbf3nvm/NVM23_train_split.txt', delimiter=' ',
                       names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])

val_df = pd.read_csv('./csbf3nvm/NVM23_val_split.txt', delimiter=' ',
                     names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='./',
    x_col='ImageFile',
    y_col=['X', 'Y', 'Z', 'W', 'P', 'Q', 'R'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=True
)

val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='./',
    x_col='ImageFile',
    y_col=['X', 'Y', 'Z', 'W', 'P', 'Q', 'R'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)

# ----------- 2. Define custom loss function -----------
def custom_loss(y_true, y_pred):
    beta = 400
    x_pred = y_pred[:, :3]
    q_pred = y_pred[:, 3:]
    x_true = y_true[:, :3]
    q_true = y_true[:, 3:]

    q_norm = K.sqrt(K.sum(K.square(q_true), axis=-1, keepdims=True)) + K.epsilon()
    q_true_normed = q_true / q_norm

    position_loss = K.mean(K.square(x_pred - x_true), axis=-1)
    quaternion_loss = K.mean(K.square(q_pred - q_true_normed), axis=-1)

    return position_loss + beta * quaternion_loss

# ----------- 3. Build CNN model (no pretrained) -----------
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    GlobalAveragePooling2D(),

    Dense(1024, activation='relu'),
    Dropout(0.5),

    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense(7)  # Output: [X, Y, Z, W, P, Q, R]
])
# ----------- 4. Compile model -----------
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=custom_loss,
              metrics=['mae'])

# ----------- 5. Custom callback (updated to log both train and val loss) -----------
class TrainValLossLogger(Callback):
    def __init__(self, train_gen, val_gen, output_file='loss_CNN_FIX.csv'):
        super().__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.output_file = output_file
        self.logs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        def compute_losses(gen):
            x, y_true = next(iter(gen))
            y_pred = self.model.predict(x)
            x_pred, q_pred = y_pred[:, :3], y_pred[:, 3:]
            x_true, q_true = y_true[:, :3], y_true[:, 3:]
            q_true_normed = q_true / (np.linalg.norm(q_true, axis=1, keepdims=True) + 1e-7)
            pos_loss = np.mean(np.square(x_pred - x_true))
            quat_loss = np.mean(np.square(q_pred - q_true_normed))
            total_loss = pos_loss + 400. * quat_loss
            return total_loss, pos_loss, quat_loss

        train_total, train_pos, train_quat = compute_losses(self.train_gen)
        val_total, val_pos, val_quat = compute_losses(self.val_gen)

        self.logs.append({
            'Epoch': epoch + 1,
            'Train_loss': train_total,
            'Train_pos_loss': train_pos,
            'Train_quat_loss': train_quat,
            'Val_loss': val_total,
            'Val_pos_loss': val_pos,
            'Val_quat_loss': val_quat
        })

        pd.DataFrame(self.logs).to_csv(self.output_file, index=False)
        print(f"Epoch {epoch+1}: train_loss={train_total:.4f}, val_loss={val_total:.4f}")

# ----------- 6. Callbacks -----------
loss_logger = TrainValLossLogger(train_generator, val_generator)
checkpoint = ModelCheckpoint('best_CNN_FIX_model.h5', monitor='val_loss',save_best_only=True,mode='min',verbose=1)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-8, verbose=1)

# ----------- 7. Train with validation -----------
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=200,
    callbacks=[checkpoint,loss_logger]
)

GPU(s) detected: ['/physical_device:GPU:0']
Found 4262 validated image filenames.
Found 1066 validated image filenames.
Epoch 1/200
134/134 [==============================] - ETA: 0s - loss: 64.3457 - mae: 0.8493
Epoch 1: val_loss improved from inf to 54.86045, saving model to best_CNN_FIX_model.h5
1/1 [==============================] - 0s 22ms/step
Epoch 1: train_loss=65.4874, val_loss=46.7905
134/134 [==============================] - 110s 620ms/step - loss: 64.3457 - mae: 0.8493 - val_loss: 54.8605 - val_mae: 0.8342
Epoch 2/200
134/134 [==============================] - ETA: 0s - loss: 56.7969 - mae: 0.8350
Epoch 2: val_loss improved from 54.86045 to 51.56994, saving model to best_CNN_FIX_model.h5
1/1 [==============================] - 0s 22ms/step
Epoch 2: train_loss=41.9852, val_loss=50.4078
134/134 [==============================] - 15s 112ms/step - loss: 56.7969 - mae: 0.8350 - val_loss: 51.5699 - val_mae: 0.8266
Epoch 3/200
134/134 [==============================] - ETA: 0s - l

In [ ]:
import pandas as pd
import numpy as np
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import tensorflow as tf

# ----------- GPU Configuration -----------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU(s) detected: {[gpu.name for gpu in gpus]}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Training on CPU.")

# ----------- 1. Load all data for training -----------
train_df = pd.read_csv('./csbf3nvm/NVM23_train_split.txt', delimiter=' ',
                       names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])

val_df = pd.read_csv('./csbf3nvm/NVM23_val_split.txt', delimiter=' ',
                     names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='./',
    x_col='ImageFile',
    y_col=['X', 'Y', 'Z', 'W', 'P', 'Q', 'R'],
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=True
)

val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='./',
    x_col='ImageFile',
    y_col=['X', 'Y', 'Z', 'W', 'P', 'Q', 'R'],
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=False
)

# ----------- 2. Define custom loss function -----------
def custom_loss(y_true, y_pred):
    beta = 400.0
    x_pred = y_pred[:, :3]
    q_pred = y_pred[:, 3:]
    x_true = y_true[:, :3]
    q_true = y_true[:, 3:]

    q_norm = K.sqrt(K.sum(K.square(q_true), axis=-1, keepdims=True)) + K.epsilon()
    q_true_normed = q_true / q_norm

    position_loss = K.mean(K.square(x_pred - x_true), axis=-1)
    quaternion_loss = K.mean(K.square(q_pred - q_true_normed), axis=-1)

    return position_loss + beta * quaternion_loss

# ----------- 3. Build model with MobileNetV3 -----------
base_model = MobileNetV3Large(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = True  # Set to True later for fine-tuning

inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=True)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(7)(x)  # Output: [X, Y, Z, W, P, Q, R]

model = Model(inputs, outputs)

# ----------- 4. Compile model -----------
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=custom_loss,
              metrics=['mae'])
model.summary()
# ----------- 5. Custom callback (updated to log both train and val loss) -----------
class TrainValLossLogger(Callback):
    def __init__(self, train_gen, val_gen, output_file='loss_mobile.csv'):
        super().__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.output_file = output_file
        self.logs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        def compute_losses(gen):
            x, y_true = next(iter(gen))
            y_pred = self.model.predict(x)
            x_pred, q_pred = y_pred[:, :3], y_pred[:, 3:]
            x_true, q_true = y_true[:, :3], y_true[:, 3:]
            q_true_normed = q_true / (np.linalg.norm(q_true, axis=1, keepdims=True) + 1e-7)
            pos_loss = np.mean(np.square(x_pred - x_true))
            quat_loss = np.mean(np.square(q_pred - q_true_normed))
            total_loss = pos_loss + 400. * quat_loss
            return total_loss, pos_loss, quat_loss

        train_total, train_pos, train_quat = compute_losses(self.train_gen)
        val_total, val_pos, val_quat = compute_losses(self.val_gen)

        self.logs.append({
            'Epoch': epoch + 1,
            'Train_loss': train_total,
            'Train_pos_loss': train_pos,
            'Train_quat_loss': train_quat,
            'Val_loss': val_total,
            'Val_pos_loss': val_pos,
            'Val_quat_loss': val_quat
        })

        pd.DataFrame(self.logs).to_csv(self.output_file, index=False)
        print(f"Epoch {epoch+1}: train_loss={train_total:.4f}, val_loss={val_total:.4f}")

# ----------- 6. Callbacks -----------
loss_logger = TrainValLossLogger(train_generator, val_generator)
# ----------- 7. Train with validation -----------
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=200,
    callbacks=[loss_logger]
)


GPU(s) detected: ['/physical_device:GPU:0']
Found 4262 validated image filenames.
Found 1066 validated image filenames.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 960)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              984064    
                                                                 
 dropout (Dropout)           (None, 1024)              0         
       

In [ ]:
import pandas as pd
import numpy as np
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import tensorflow as tf

# ----------- GPU Configuration -----------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU(s) detected: {[gpu.name for gpu in gpus]}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Training on CPU.")

# ----------- 1. Load all data for training -----------
train_df = pd.read_csv('./csbf3nvm/NVM23_train_split.txt', delimiter=' ',
                       names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])

val_df = pd.read_csv('./csbf3nvm/NVM23_val_split.txt', delimiter=' ',
                     names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='./',
    x_col='ImageFile',
    y_col=['X', 'Y', 'Z', 'W', 'P', 'Q', 'R'],
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=True
)

val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='./',
    x_col='ImageFile',
    y_col=['X', 'Y', 'Z', 'W', 'P', 'Q', 'R'],
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=False
)

# ----------- 2. Define custom loss function -----------
def custom_loss(y_true, y_pred):
    beta = 400.0
    x_pred = y_pred[:, :3]
    q_pred = y_pred[:, 3:]
    x_true = y_true[:, :3]
    q_true = y_true[:, 3:]

    q_norm = K.sqrt(K.sum(K.square(q_true), axis=-1, keepdims=True)) + K.epsilon()
    q_true_normed = q_true / q_norm

    position_loss = K.mean(K.square(x_pred - x_true), axis=-1)
    quaternion_loss = K.mean(K.square(q_pred - q_true_normed), axis=-1)

    return position_loss + beta * quaternion_loss

# ----------- 3. Build EfficientNetV2B0 model -----------
base_model = EfficientNetV2B0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(7)(x)

model = Model(inputs=base_model.input, outputs=predictions)

# ----------- 4. Compile model -----------
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=custom_loss,
              metrics=['mae'])
model.summary()
# ----------- 5. Custom callback (updated to log both train and val loss) -----------
class TrainValLossLogger(Callback):
    def __init__(self, train_gen, val_gen, output_file='loss_EfficientNetV2B0.csv'):
        super().__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.output_file = output_file
        self.logs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        def compute_losses(gen):
            x, y_true = next(iter(gen))
            y_pred = self.model.predict(x)
            x_pred, q_pred = y_pred[:, :3], y_pred[:, 3:]
            x_true, q_true = y_true[:, :3], y_true[:, 3:]
            q_true_normed = q_true / (np.linalg.norm(q_true, axis=1, keepdims=True) + 1e-7)
            pos_loss = np.mean(np.square(x_pred - x_true))
            quat_loss = np.mean(np.square(q_pred - q_true_normed))
            total_loss = pos_loss + 400. * quat_loss
            return total_loss, pos_loss, quat_loss

        train_total, train_pos, train_quat = compute_losses(self.train_gen)
        val_total, val_pos, val_quat = compute_losses(self.val_gen)

        self.logs.append({
            'Epoch': epoch + 1,
            'Train_loss': train_total,
            'Train_pos_loss': train_pos,
            'Train_quat_loss': train_quat,
            'Val_loss': val_total,
            'Val_pos_loss': val_pos,
            'Val_quat_loss': val_quat
        })

        pd.DataFrame(self.logs).to_csv(self.output_file, index=False)
        print(f"Epoch {epoch+1}: train_loss={train_total:.4f}, val_loss={val_total:.4f}")

# ----------- 6. Callbacks (Remove checkpoint) -----------
loss_logger = TrainValLossLogger(train_generator, val_generator)

# ----------- 7. Train with validation -----------
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=200,
    callbacks=[loss_logger]
)

# ----------- 8. Save final model (last epoch) -----------
model.save('final_model_EfficientNetV2B0.h5')

GPU(s) detected: ['/physical_device:GPU:0']
Found 4262 validated image filenames.
Found 1066 validated image filenames.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  0           ['rescaling[0][0]']              
                                                                         

In [ ]:
import pandas as pd
import numpy as np
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import tensorflow as tf

# ----------- GPU Configuration -----------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU(s) detected: {[gpu.name for gpu in gpus]}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Training on CPU.")

# ----------- 1. Load all data for training -----------
train_df = pd.read_csv('./csbf3nvm/NVM23_train_split.txt', delimiter=' ',
                       names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])

val_df = pd.read_csv('./csbf3nvm/NVM23_val_split.txt', delimiter=' ',
                     names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='./',
    x_col='ImageFile',
    y_col=['X', 'Y', 'Z', 'W', 'P', 'Q', 'R'],
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=True
)

val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='./',
    x_col='ImageFile',
    y_col=['X', 'Y', 'Z', 'W', 'P', 'Q', 'R'],
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=False
)

# ----------- 2. Define custom loss function -----------
def custom_loss(y_true, y_pred):
    beta = 400.0
    x_pred = y_pred[:, :3]
    q_pred = y_pred[:, 3:]
    x_true = y_true[:, :3]
    q_true = y_true[:, 3:]

    q_norm = K.sqrt(K.sum(K.square(q_true), axis=-1, keepdims=True)) + K.epsilon()
    q_true_normed = q_true / q_norm

    position_loss = K.mean(K.square(x_pred - x_true), axis=-1)
    quaternion_loss = K.mean(K.square(q_pred - q_true_normed), axis=-1)

    return position_loss + beta * quaternion_loss

# ----------- 3. Build InceptionV3 model -----------
base_model = InceptionV3(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

x = base_model.output
x = Flatten()(x) 
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(7)(x)

model = Model(inputs=base_model.input, outputs=predictions)

# ----------- 4. Compile model -----------
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=custom_loss,
              metrics=['mae'])
model.summary()
class TrainValLossLogger(Callback):
    def __init__(self, train_gen, val_gen, output_file='loss_InceptionV3_.csv'):
        super().__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.output_file = output_file
        self.logs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        def compute_losses(gen):
            x, y_true = next(iter(gen))
            y_pred = self.model.predict(x)
            x_pred, q_pred = y_pred[:, :3], y_pred[:, 3:]
            x_true, q_true = y_true[:, :3], y_true[:, 3:]
            q_true_normed = q_true / (np.linalg.norm(q_true, axis=1, keepdims=True) + 1e-7)
            pos_loss = np.mean(np.square(x_pred - x_true))
            quat_loss = np.mean(np.square(q_pred - q_true_normed))
            total_loss = pos_loss + 400. * quat_loss
            return total_loss, pos_loss, quat_loss

        train_total, train_pos, train_quat = compute_losses(self.train_gen)
        val_total, val_pos, val_quat = compute_losses(self.val_gen)

        self.logs.append({
            'Epoch': epoch + 1,
            'Train_loss': train_total,
            'Train_pos_loss': train_pos,
            'Train_quat_loss': train_quat,
            'Val_loss': val_total,
            'Val_pos_loss': val_pos,
            'Val_quat_loss': val_quat
        })

        pd.DataFrame(self.logs).to_csv(self.output_file, index=False)
        print(f"Epoch {epoch+1}: train_loss={train_total:.4f}, val_loss={val_total:.4f}")

# ----------- 6. Callbacks -----------
loss_logger = TrainValLossLogger(train_generator, val_generator)

# ----------- 7. Train with validation -----------
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=200,
    callbacks=[loss_logger]
)

model.save('final_model_InceptionV3_.h5')

# Validate

In [1]:
import tensorflow as tf

In [2]:
def custom_loss(y_true, y_pred):
    beta = 400
    x_pred = y_pred[:, :3]
    q_pred = y_pred[:, 3:]
    x_true = y_true[:, :3]
    q_true = y_true[:, 3:]
    
    q_norm = K.sqrt(K.sum(K.square(q_true), axis=-1, keepdims=True))
    q_true_normed = q_true / q_norm
    
    return K.mean(K.square(x_pred - x_true), axis=-1) + beta * K.mean(K.square(q_pred - q_true_normed), axis=-1)

In [4]:
model = tf.keras.models.load_model('best_CNN_FIX_model.h5', custom_objects={'custom_loss': custom_loss})

In [7]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# test_df = pd.read_csv('./csbf3nvm/NVM23_test_stair.txt', delimiter=' ', skiprows=1, names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])
# test_df[['X', 'Y', 'Z', 'W', 'P', 'Q', 'R']] = test_df[['X', 'Y', 'Z', 'W', 'P', 'Q', 'R']] 
test_df = pd.read_csv('csbf3nvm/NVM23_test.txt', delimiter=' ',
                   names=['ImageFile', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R'])

datagen = ImageDataGenerator(rescale=1./255)

test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='./',
    x_col='ImageFile',
    y_col=['X', 'Y', 'Z', 'W', 'P', 'Q', 'R'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)

# Get predictions
predictions = model.predict(test_generator, steps=len(test_df))

# Combine actual and predicted values
results = test_df.copy()
results[['Pred_X', 'Pred_Y', 'Pred_Z', 'Pred_W', 'Pred_P', 'Pred_Q', 'Pred_R']] = predictions

# Save to CSV
results.to_csv('./DeepNav_test_score.csv', index=False)

# Print Mean Squared Error
mse = np.mean((test_df[['X', 'Y', 'Z', 'W', 'P', 'Q', 'R']].values - predictions)**2)
print(f"Mean Squared Error: {mse}")

# Print example outputs for verification
for i, row in results.iterrows():
    print(f"Image: {row['ImageFile']}")
    print(f"Actual   : {row[['X', 'Y', 'Z', 'W', 'P', 'Q', 'R']].values}")
    print(f"Predicted: {row[['Pred_X', 'Pred_Y', 'Pred_Z', 'Pred_W', 'Pred_P', 'Pred_Q', 'Pred_R']].values}\n")


Found 592 validated image filenames.
592/592 [==============================] - 4s 1ms/step
Mean Squared Error: 0.06549714834953776
Image: seq_val/S_2_3_116_0.jpg
Actual   : [0.0212611942306 -0.150198364279 -4.02815241156 0.992784597423
 -0.0231174873332 0.116162444327 0.0187246209027]
Predicted: [-0.35580921173095703 -0.11551351845264435 -4.122383117675781
 0.9832386374473572 -0.006837533786892891 0.07170430570840836
 0.022308899089694023]

Image: seq_val/IMG_9804_0.jpg
Actual   : [-3.66592251994 1.02639878824 2.82205127708 0.949369820332 0.0210068530886
 -0.313457549781 -0.000144397914525]
Predicted: [-3.2417845726013184 0.7269228100776672 2.040851593017578
 0.930647075176239 -0.0058696214109659195 -0.28249043226242065
 -7.445248775184155e-05]

Image: seq_val/IMG_9862_0.jpg
Actual   : [4.38230986374 0.392425569864 3.84121422039 0.452808276381
 -0.0231972306729 -0.89107694329 -0.0202097616994]
Predicted: [4.204751014709473 0.4122766852378845 3.631880044937134 0.4574793875217438
 -0.03

In [8]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# Load the uploaded CSV again
csv_path = "./DeepNav_test_score.csv"
df = pd.read_csv(csv_path)


# Define true and predicted column names based on actual file structure
true_cols = ['X', 'Y', 'Z', 'W', 'P', 'Q', 'R']
pred_cols = ['Pred_X', 'Pred_Y', 'Pred_Z', 'Pred_W', 'Pred_P', 'Pred_Q', 'Pred_R']

y_true = df[true_cols].values
y_pred = df[pred_cols].values

# Compute MSE
mse_xyz = mean_squared_error(y_true[:, :3], y_pred[:, :3])
mse_wpqr = mean_squared_error(y_true[:, 3:], y_pred[:, 3:])
custom_loss = mse_xyz + 400 * mse_wpqr
custom_loss = round(custom_loss, 6)
mse_xyz = round(mse_xyz, 6)
mse_wpqr = round(mse_wpqr, 6)

custom_loss, mse_xyz, mse_wpqr


(0.418182, 0.151939, 0.000666)